# Analysis single stock market price using yfinance module

In [ ]:
# Upgrade the module 
# Ref: https://stackoverflow.com/questions/68320184/yfinance-returning-error-when-downloading-data
#!pip install yfinance --upgrade --no-cache-dir

In [1]:
import yfinance as yf
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set any float number precision 2 decimal points
pd.set_option('precision', 2)

In [2]:
start_date = '2016-01-01'
end_date = '2021-09-25'

In [3]:
# Extract stock price only 'Adj Close' and 'Volume'

df = yf.download('BTC-USD',start=start_date,end=end_date,interval='1d',rounding=1)[['Adj Close', 'Volume']]#.to_frame()

[*********************100%***********************]  1 of 1 completed


In [40]:
df.tail(5)

,Adj Close,Volume,AdjC_cum,AdjC_pct
Date,,,,
2021-09-20,42843.80,43909845642,42413.23,-9.34
2021-09-21,40693.68,48701090088,40263.11,-5.02
2021-09-22,43574.51,38139709246,43143.94,7.08
2021-09-23,44895.10,34244064430,44464.53,3.03
2021-09-24,42839.75,42839345714,42409.18,-4.58


In [5]:
# Define function for style
def above_zero(val):
  color = 'green' if val > 0 else 'red'
  return 'color: %s' % color

In [6]:
# Copy file to prevent A value is trying to be set on a copy of a slice from a DataFrame
df = df.copy()

# Find Adjusted Close price difference and cumulative sum
df.loc[:,'AdjC_cum'] = df.loc[:,'Adj Close'] - df.loc[:,'Adj Close'].shift(periods = 1)
df.loc[:,'AdjC_cum'] = df.loc[:,'AdjC_cum'].cumsum()

# Similar method to Find Adjusted Close price difference and cumulative sum in a single command diff
#df.loc[:,'AdjC_cum1'] = df.loc[:,'Adj Close'].diff().cumsum()

# Find Adjusted Close price percentage
df.loc[:,'AdjC_pct'] = df.loc[:,'Adj Close'].pct_change()*100

# Fill NaN value with zero
df.fillna(value = 0, inplace = True)

In [36]:
df.shape

(2091, 4)

In [33]:
df['AdjC_pct'].value_counts(bins = 20).to_frame().nlargest(5,'AdjC_pct')

,AdjC_pct
"(-2.84, 0.28]",777
"(0.28, 3.401]",699
"(3.401, 6.522]",205
"(-5.961, -2.84]",185
"(-9.082, -5.961]",71


In [38]:
df['Volume'].value_counts(bins = 50).to_frame().nlargest(5,'Volume')

,Volume
"(-322425427.47999996, 7047302549.58]",1028
"(14066091099.16, 21084879648.74]",226
"(7047302549.58, 14066091099.16]",183
"(21084879648.74, 28103668198.32]",181
"(28103668198.32, 35122456747.9]",137


In [69]:
# Highlight column 'AdjC_pct' and put bar on 'Volume'
df.sample(3).style.applymap(above_zero, subset=['AdjC_pct'])\
.bar(subset = ['Volume'])\
.format({'Adj Close': "{:.2f}",'AdjC_pct': "{:.2f}",'AdjC_cum': "{:.2f}"})\
.highlight_max()

,Adj Close,Volume,AdjC_cum,AdjC_pct
Date,,,,
2016-06-25 00:00:00,665.12,126656000,234.55,-0.03
2016-11-07 00:00:00,703.13,65047100,272.56,-1.18
2018-12-26 00:00:00,3857.30,5326547918,3426.73,1.10


In [67]:
# Find out when was the highest volume transaction
df.loc[df['Volume'].idxmax()]

# Find out when was the lowest volume transaction
df.loc[df['Volume'].idxmin()]

# Find out when was the highest AdjC_pct occur
df.loc[df['AdjC_pct'].idxmax()]

# Find out when was the lowest AdjC_pct occur
df.loc[df['AdjC_pct'].idxmin()]

Adj Close    4.97e+03
Volume       5.40e+10
AdjC_cum     4.54e+03
AdjC_pct    -3.72e+01
Name: 2020-03-12 00:00:00, dtype: float64

<div class="alert alert-block alert-info">
<b>Key insight based on daily basis transaction (2,091 data):<br/> 
1. Typical BTC price % change is between <font color='black'>-2.8% to 3.4%</font>.<br/>
    2. Highest BTC price % change is <font color='black'>25.2%</font> occur on Dec 12th, 2017.<br/>
    3. Lowest BTC price % change is <font color='black'>-37.2%</font> occur on Mar 12th, 2020.<br/>        
4. Typical volume transaction spans from <font color='black'>-300 million USD to 7 billion USD</font>.<br/>
5. Highest volume transaction is <font color='black'>350 billion USD</font> occur on Feb 26th, 2021.<br/>
6. Lowest volume transaction is <font color='black'>28 million USD</font> occur on May 5th, 2016.<br/>   
</div>

In [80]:
# Create list of days category
dayscat = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create list of months category
monthscat = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

### Check typical % change in monthly basis

In [81]:
# Groupby months name → find average → highlight on volume
df.groupby(df.index.month_name()).mean().reindex(monthscat).style.applymap(above_zero, subset=['AdjC_pct'])\
.format({'Adj Close': "{:.2f}",\
         'Volume': "{:.0f}",\
         'AdjC_pct': "{:.2f}",\
         'AdjC_cum': "{:.2f}"})\
.bar(subset = ['Volume'])

,Adj Close,Volume,AdjC_cum,AdjC_pct
Date,,,,
January,10200.87,19352123816,9770.30,0.02
February,11685.35,22738275797,11254.78,0.50
March,12738.59,18691054476,12308.02,-0.15
April,13248.22,20232284940,12817.65,0.68
May,12304.08,22694771238,11873.51,0.42
June,10803.81,14976989882,10373.24,0.26
July,10838.56,11929296398,10407.99,0.39
August,13194.23,13201952984,12763.66,0.27
September,11932.82,14245875997,11502.25,-0.17


In [133]:
# Set pivot table based segregate based on year and month name, then calculate the mean aggregate
df2 = df.pivot_table(index=df.index.year,
                     columns = df.index.month_name(),
                     fill_value = 0,
                     aggfunc = {'AdjC_pct':'mean'})

# Remove the first multi index column 
df2.columns = df2.columns.droplevel(0)

# Re-index based on monthscat
df2 = df2.reindex(columns=monthscat)

# Apply style
df2.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,January,February,March,April,May,June,July,August,September,October,November,December
Date,,,,,,,,,,,,
2015,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2016,-0.41,0.61,-0.15,0.25,0.58,0.90,-0.21,-0.24,0.20,0.46,0.23,0.85
2017,0.15,0.72,-0.19,0.78,1.81,0.36,0.67,1.67,-0.06,1.36,1.64,1.41
2018,-0.84,0.31,-1.18,1.04,-0.64,-0.46,0.68,-0.29,-0.17,-0.14,-1.38,-0.13
2019,-0.22,0.42,0.21,0.95,1.63,0.92,-0.09,-0.10,-0.46,0.40,-0.62,-0.14
2020,0.89,-0.26,-0.48,1.09,0.34,-0.09,0.72,0.13,-0.22,0.90,1.25,1.32
2021,0.58,1.24,0.93,-0.00,-1.23,-0.08,0.60,0.46,-0.31,0.00,0.00,0.00


In [134]:
# Apply style in transpose to reconfirm the pattern
df2.T.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,2015,2016,2017,2018,2019,2020,2021
Date,,,,,,,
January,0.00,-0.41,0.15,-0.84,-0.22,0.89,0.58
February,0.00,0.61,0.72,0.31,0.42,-0.26,1.24
March,0.00,-0.15,-0.19,-1.18,0.21,-0.48,0.93
April,0.00,0.25,0.78,1.04,0.95,1.09,-0.00
May,0.00,0.58,1.81,-0.64,1.63,0.34,-1.23
June,0.00,0.90,0.36,-0.46,0.92,-0.09,-0.08
July,0.00,-0.21,0.67,0.68,-0.09,0.72,0.60
August,0.00,-0.24,1.67,-0.29,-0.10,0.13,0.46
September,0.00,0.20,-0.06,-0.17,-0.46,-0.22,-0.31


<div class="alert alert-block alert-info">
<b>Key insight:<br/> 
1. <font color='green'>March and September</font> is the month with lowest % price change.<br/>
2. <font color='red'>April and July</font> is the month with highest % price change.<br/>
3. BTC volume transaction typically active at the first half of the year.<br/>
4. No common pattern seen from BTC transaction since 2016 up to 2021.<br/> 
5. <font color='red'>BTC price % pattern start change since 2020 onward</font>.<br/>
</div>

### Check typical % change in daily basis

In [85]:
# Groupby days name → find average → highlight on volume → set precision
df.groupby(df.index.day_name()).mean().reindex(dayscat).style\
.bar(subset = ['Volume'])\
.format({'Adj Close': "{:.2f}",\
         'Volume': "{:.0f}",\
         'AdjC_pct': "{:.2f}",\
         'AdjC_cum': "{:.2f}"})

,Adj Close,Volume,AdjC_cum,AdjC_pct
Date,,,,
Monday,11023.44,16879343629,10592.87,0.57
Tuesday,11006.03,16847867596,10575.46,0.15
Wednesday,11064.96,17361362502,10634.39,0.42
Thursday,11028.69,17411253361,10598.12,0.15
Friday,11106.93,17844854425,10676.36,0.34
Saturday,11021.08,14856512865,10590.51,0.45
Sunday,11008.21,14818629742,10577.64,0.02


In [88]:
# Set pivot table based segregate based on year and day name, then calculate the mean aggregate
df3 = df.pivot_table(index=df.index.year,
                     columns = df.index.day_name(),
                     fill_value = 0,
                     aggfunc = {'AdjC_pct':'mean'})

# Remove the first multi index column 
df3.columns = df3.columns.droplevel(0)

# Re-index based on dayscat
df3 = df3.reindex(columns=dayscat)

# Fill NaN value with zero
df3.fillna(value = 0, inplace = True)

# Apply style
df3.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Date,,,,,,,
2015,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2016,0.20,-0.21,0.36,0.42,0.29,0.64,0.05
2017,2.10,1.12,0.65,1.28,-0.29,0.64,0.54
2018,-0.66,-0.34,-0.81,-0.22,0.00,0.48,-0.36
2019,0.58,-0.23,0.56,-0.81,1.08,0.46,0.06
2020,1.21,0.68,0.92,0.17,0.30,0.30,-0.33
2021,-0.16,-0.20,1.00,0.06,0.76,0.09,0.18


In [119]:
# Apply style in transpose to reconfirm the pattern
df3.T.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,2015,2016,2017,2018,2019,2020,2021
Date,,,,,,,
Monday,0.00,0.20,2.10,-0.66,0.58,1.21,-0.16
Tuesday,0.00,-0.21,1.12,-0.34,-0.23,0.68,-0.20
Wednesday,0.00,0.36,0.65,-0.81,0.56,0.92,1.00
Thursday,0.00,0.42,1.28,-0.22,-0.81,0.17,0.06
Friday,0.00,0.29,-0.29,0.00,1.08,0.30,0.76
Saturday,0.00,0.64,0.64,0.48,0.46,0.30,0.09
Sunday,0.00,0.05,0.54,-0.36,0.06,-0.33,0.18


### Check typical % change in daily basis in each month

In [117]:
# Set pivot table based segregate based on month and day name, then calculate the mean aggregate
df4 = df.pivot_table(index=df.index.month_name(),
                     columns = df.index.day_name(),
                     fill_value = 0,
                     aggfunc = {'AdjC_pct':'mean'})

# Remove the first multi index column 
df4.columns = df4.columns.droplevel(0)

# Re-index based on monthscat
df4 = df4.T.reindex(columns=monthscat)

# Re-index based on dayscat
df4 = df4.T.reindex(columns=dayscat)

# Apply style
df4.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Date,,,,,,,
January,-0.77,0.32,0.88,-1.07,0.36,1.11,-0.69
February,0.91,0.93,0.41,0.16,1.31,0.55,-0.76
March,0.90,0.50,-0.68,-1.23,-0.15,-0.62,0.30
April,0.75,2.03,-0.16,1.15,0.17,0.53,0.29
May,1.15,-0.04,0.37,0.82,-0.53,1.02,0.13
June,0.22,-0.25,0.08,0.56,0.51,0.49,0.20
July,1.80,-0.23,0.98,0.22,0.37,-0.02,-0.37
August,0.88,-1.42,-0.18,0.85,1.26,0.69,-0.15
September,0.09,-0.85,0.14,-0.96,0.30,0.22,-0.14


In [118]:
# Apply style in transpose to reconfirm the pattern
df4.T.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,January,February,March,April,May,June,July,August,September,October,November,December
Date,,,,,,,,,,,,
Monday,-0.77,0.91,0.90,0.75,1.15,0.22,1.80,0.88,0.09,0.45,0.07,0.11
Tuesday,0.32,0.93,0.50,2.03,-0.04,-0.25,-0.23,-1.42,-0.85,0.00,0.78,0.20
Wednesday,0.88,0.41,-0.68,-0.16,0.37,0.08,0.98,-0.18,0.14,0.06,1.80,1.81
Thursday,-1.07,0.16,-1.23,1.15,0.82,0.56,0.22,0.85,-0.96,0.86,-0.41,1.01
Friday,0.36,1.31,-0.15,0.17,-0.53,0.51,0.37,1.26,0.30,0.96,-0.58,0.16
Saturday,1.11,0.55,-0.62,0.53,1.02,0.49,-0.02,0.69,0.22,1.17,-0.46,0.74
Sunday,-0.69,-0.76,0.30,0.29,0.13,0.20,-0.37,-0.15,-0.14,0.64,0.36,0.62


<div class="alert alert-block alert-info">
<b>Key insight:<br/> 
1. <font color='green'>Sunday</font> is the lowest % price change.<br/>
2. Typically BTC volume transaction quite active during <font color='red'>weekdays</font> and slower on <font color='green'>weekend</font>.<br/>
</div>

### Check typical % change in date basis

In [131]:
# Groupby day name → find average → highlight on volume → set precision
df.groupby(df.index.day).mean().style.applymap(above_zero, subset=['AdjC_pct'])\
.bar(subset = ['Volume'])\
.format({'Adj Close': "{:.2f}",\
         'Volume': "{:.0f}",\
         'AdjC_pct': "{:.2f}",\
         'AdjC_cum': "{:.2f}"})\
.highlight_max()

,Adj Close,Volume,AdjC_cum,AdjC_pct
Date,,,,
1,10770.03,15076459516,10339.46,0.87
2,10859.65,15776371434,10429.08,0.89
3,10983.33,15693803410,10552.76,0.76
4,10862.90,15755327346,10432.33,-0.19
5,11049.78,15836977735,10619.21,0.71
6,11176.84,16410939809,10746.27,0.87
7,11169.70,17040328138,10739.13,0.36
8,11286.08,17219016575,10855.51,0.34
9,11400.03,16247922171,10969.46,0.14


In [123]:
# Set pivot table based segregate based on year and date, then calculate the mean aggregate
df5 = df.pivot_table(index=df.index.year,
                     columns = df.index.day,
                     fill_value = 0,
                     aggfunc = {'AdjC_pct':'mean'})

# Remove the first multi index column 
df5.columns = df5.columns.droplevel(0)

# Apply style
df5.T.style.applymap(above_zero).background_gradient(cmap='RdYlGn_r')

Date,2015,2016,2017,2018,2019,2020,2021
Date,,,,,,,
1,0.00,0.57,2.56,-0.48,0.62,1.15,0.74
2,0.00,-0.36,1.37,1.26,2.85,-1.11,1.51
3,0.00,-0.18,1.76,0.86,0.79,-0.15,1.71
4,0.00,1.10,1.08,-1.51,-1.05,0.36,-1.41
5,0.00,-0.55,1.27,-0.95,1.11,1.80,1.89
6,0.00,0.72,1.96,-0.86,0.09,1.69,1.86
7,0.00,0.21,2.10,-1.36,1.31,0.30,-0.65
8,0.00,0.47,0.11,-1.34,1.06,-0.42,2.73
9,0.00,-0.25,-1.17,0.43,0.25,-0.52,2.67


<div class="alert alert-block alert-info">
<b>Key insight:<br/> 
1. <font color='red'>Day 20</font> is the highest % price change.<br/>
2. <font color='green'>Day 10 and Day 21</font> is the lowest % price change.<br/>
3. <font color='magenta'>Day 26</font> is the day with highest volume transaction.<br/>
</div>

### Key Takeaway

<div class="alert alert-block alert-success">

General finding:<br/>
1. Typical BTC price % change is between <font color='black'>-2.8% to 3.4%</font>.<br/>
2. Highest BTC price % change is <font color='black'>25.2%</font> occur on Dec 12th, 2017.<br/>
3. Lowest BTC price % change is <font color='black'>-37.2%</font> occur on Mar 12th, 2020.<br/>        
4. Typical volume transaction spans from <font color='black'>-300 million USD to 7 billion USD</font>.<br/>
5. Highest volume transaction is <font color='black'>350 billion USD</font> occur on Feb 26th, 2021.<br/>
6. Lowest volume transaction is <font color='black'>28 million USD</font> occur on May 5th, 2016.<br/>   

From month perspective:<br/>
7. <font color='blue'>March and September</font> is the month with lowest % price change.<br/>
8. <font color='red'>April and July</font> is the month with highest % price change.<br/>
9. BTC volume transaction typically active at the first half of the year.<br/>
10. No common pattern seen from BTC transaction since 2016 up to 2021.<br/> 
11. <font color='red'>BTC price % pattern start change since 2020 onward</font>.<br/>

From day perspective:<br/>

12. <font color='blue'>Sunday</font> is the lowest % price change.<br/>
13. Typically BTC volume transaction quite active during <font color='red'>weekdays</font> and slower on <font color='green'>weekend</font>.<br/>

From date perspective:<br/>
14. <font color='red'>Day 20</font> is the highest % price change.<br/>
15. <font color='blue'>Day 10 and Day 21</font> is the lowest % price change.<br/>
16. <font color='magenta'>Day 26</font> is the day with highest volume transaction.<br/>
</div>